In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np

In [2]:
#load data 
df = pd.read_csv('/Users/sa21/Desktop/Podcast_Prediction/Data/processed/df_train_cleaned.csv')


In [3]:
df.head()

,id,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,Genre_Business,Genre_Comedy,Genre_Education,...,Publication_Day_Monday,Publication_Day_Saturday,Publication_Day_Sunday,Publication_Day_Thursday,Publication_Day_Tuesday,Publication_Day_Wednesday,Publication_Time_Afternoon,Publication_Time_Evening,Publication_Time_Morning,Publication_Time_Night
0,0,63.84,74.81,53.38,0.0,1,31.41998,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,1,119.80,66.95,75.95,2.0,-1,88.01241,0,1,0,...,0,1,0,0,0,0,1,0,0,0
2,2,73.90,69.97,8.97,0.0,-1,44.92531,0,0,1,...,0,0,0,0,1,0,0,1,0,0
3,3,67.17,57.22,78.70,2.0,1,46.27824,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,4,110.51,80.07,58.68,3.0,0,75.61031,0,0,0,...,1,0,0,0,0,0,1,0,0,0


# RandomForestRegressor with default hyperparameters

In [4]:
# Save target and features
X = df.drop(columns=['Listening_Time_minutes'])
y = df['Listening_Time_minutes']
# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# Extract validation 'id' BEFORE dropping 'id'
id_val = X_val['id']  # these match y_pred
X_train = X_train.drop(columns=['id'])  # safe to drop here
X_val = X_val.drop(columns=['id'])

#Train 
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_val)

# Evaluate
print("MSE:", mean_squared_error(y_val, y_pred))
print("R² Score:", r2_score(y_val, y_pred))

# Save predictions with id and Listening_Time_minutes
df_results = pd.DataFrame({
    'id': id_val.values,  # ensure this is same length as y_pred
    'Listening_Time_minutes': y_pred
})

df_results.to_csv('/Users/sa21/Desktop/Podcast_Prediction/Data/predictions/Random_Forest_default_predictions.csv', index=False)



MSE: 163.44457625723746
R² Score: 0.7778764551237489


# RandomForestRegressor with Randomized Search

In [5]:
# SAMPLE of training data for tuning
X_sample = X_train.sample(frac=0.15, random_state=42)
y_sample = y_train.loc[X_sample.index]

# Randomized parameter grid
param_dist = {
    'n_estimators': randint(100, 300),
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': randint(2, 10)
}

# Setup RandomizedSearchCV with fewer samples
random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=10,
    cv=2,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

# Fit on sample
random_search.fit(X_sample, y_sample)
best_random_model = random_search.best_estimator_

# Predict on full validation set
y_pred_random = best_random_model.predict(X_val)

# Evaluate
print("RandomSearchCV Evaluation (sample tuned):")
print(f"MSE: {mean_squared_error(y_val, y_pred_random):.3f}")
print(f"R² Score: {r2_score(y_val, y_pred_random):.3f}")

# Save id Listening_Time_minutes predictions
df_random_results = pd.DataFrame({
    'id': id_val.values,
    'Listening_Time_minutes': y_pred_random
})

df_random_results.to_csv('/Users/sa21/Desktop/Podcast_Prediction/Data/predictions/Random_Forest_randomsearch_predictions.csv', index=False)



RandomSearchCV Evaluation (sample tuned):
MSE: 172.857
R² Score: 0.765


#  Predicting on Cleaned Test Data

In [6]:
# Load cleaned test data
df_test = pd.read_csv('/Users/sa21/Desktop/Podcast_Prediction/Data/processed/df_test_cleaned.csv')

# Drop ID column temporarily
test_ids = df_test['id']
X_test_final = df_test.drop(columns=['id'])

# one-hot encoding created mismatched columns
missing_cols = set(X_train.columns) - set(X_test_final.columns)
for col in missing_cols:
    X_test_final[col] = 0  # add missing columns with 0s

# Reorder to match training feature order
X_test_final = X_test_final[X_train.columns]

# Predict with trained model
y_test_pred = model.predict(X_test_final)

# Save predictions
test_results = pd.DataFrame({
    'id': test_ids,
    'Listening_Time_minutes': y_test_pred
})

test_results.to_csv('/Users/sa21/Desktop/Podcast_Prediction/Data/predictions/randomforest_test_predictions.csv', index=False)
